In [10]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn import preprocessing
pd.set_option("display.max_columns", 100)
from sklearn.metrics import f1_score
DATA_PATH = Path.cwd().parent / "ML project" 
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.impute import MissingIndicator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import Normalizer
RANDOM_SEED = 6    # Set a random seed for reproducibility!
import copy

In [11]:

from math import cos, asin, sqrt, pi

def distanceLong(lat1, lon1, lat2, lon2):
    p = pi/180
    latDiff = (lat2-lat1)*p
    lonDiff = (lon2-lon1)*p
    coslat1 = np.cos(lat1*p)
    coslat2 = np.cos(lat2*p)
    coslondiff = np.cos(lonDiff)
    coslatdiff = np.cos(latDiff)
    a = 0.5 - coslatdiff/2 + coslat1 * coslat2 * (1-coslondiff)/2
    return 12742 * (np.arcsin(a**0.5)) 

In [12]:

def formatdata(dataFrame):
    dataFrame['man_fare'] = 55
    dataFrame['pickup_time'] = dataFrame['pickup_time']+ '.099'
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x:  x.replace("/","-"))
    dataFrame['drop_time'] = dataFrame['drop_time']+ '.099'

    pd.to_datetime(dataFrame['pickup_time'], format='%m-%d-%Y %H:%M:%S.%f', errors='ignore')
    pd.to_datetime(dataFrame['drop_time'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore')
    nightHours = [21,22,23,0,1,2,3,4,5]
    dataFrame['rush'] = (pd.to_datetime(dataFrame['pickup_time'], errors='ignore').dt.hour)
    dataFrame['rush'] = dataFrame['rush'].apply(lambda x: ((x >= 7 and x < 9) or (x >= 17 and x < 20)))
    dataFrame['nightHours'] = (pd.to_datetime(dataFrame['pickup_time'], errors='ignore').dt.hour)
    dataFrame['nightHours'] = dataFrame['nightHours'].apply(lambda x: x in nightHours)
    dataFrame['pickup_time'] = dataFrame['pickup_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m-%d-%Y %H:%M.%f')))
    dataFrame['drop_time'] = dataFrame['drop_time'].apply(lambda x: datetime.timestamp(datetime.strptime(x,'%m/%d/%Y %H:%M.%f')))
    dataFrame['timediff'] =  dataFrame['drop_time'] - dataFrame['pickup_time']
    #dataFrame = dataFrame.drop(['pickup_time', 'drop_time'], axis = 1 )
    dataFrame['fareMin'] = (dataFrame['fare']- dataFrame['man_fare']) / (dataFrame['timediff']+1)
    
   #dataFrame['timediff1'] =  dataFrame['timediff']**2
    dataFrame['distance'] = distanceLong(dataFrame['drop_lat'],dataFrame['drop_lon'],dataFrame['pick_lat'],dataFrame['pick_lon'])*100
    #dataFrame['distance1'] = ((dataFrame['drop_lat']-dataFrame['pick_lat'])**2+(dataFrame['drop_lon']-dataFrame['pick_lon'])**2)**0.5
    #dataFrame = dataFrame.drop(['drop_lat', 'pick_lat', 'drop_lon', 'pick_lon'], axis=1)
    dataFrame['meterMinfare'] = dataFrame['meter_waiting_fare'] / (dataFrame['meter_waiting']+1)
    dataFrame['drivemintime'] = (dataFrame['fare'] - dataFrame['meter_waiting_fare']) / (dataFrame['timediff'] - dataFrame['meter_waiting'])
    dataFrame['fareDis'] = (dataFrame['fare']) / (dataFrame['distance']+1)
    dataFrame['speed'] = 60*dataFrame['distance'] / (dataFrame['duration']+1)
    #dataFrame['unit_meter_waiting_fair'] = dataFrame['meter_waiting_fare'] / dataFrame['meter_waiting']
    #print(dataFrame['unit_meter_waiting_fair'])
    dataFrame['allrush'] = dataFrame['rush'] + dataFrame['nightHours']
    #dataFrame['_fare'] = dataFrame['fare'] - dataFrame['additional_fare'] 
    #dataFrame['tripDuration'] = dataFrame['duration'] - dataFrame['meter_waiting']
    #print(dataFrame['fareMin'])
    
    return dataFrame

 
bestDegree = 0
bestBias = False
bestInteraction = False
bestscore = 0

In [13]:
train_df = pd.read_csv(
    DATA_PATH / "train.csv", 
    index_col="tripid"
)



train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})

t_df_copy = copy.copy(train_df)
count=0

train_df_cols = t_df_copy.columns.values
li = list(train_df_cols)
features_df = t_df_copy[li[0:-1]]
labels_df = t_df_copy[li[-1]]

features_df = formatdata(features_df)
features_df = features_df.fillna(features_df.mean())
numeric_cols = features_df.columns[features_df.dtypes != object].values


C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:178: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [14]:

solver = 'lbfgs'
alpha = 0.2
hidden_layer_sizes=(10,2 )
random_state=2
max_iter= 10000000
activation='tanh'
estimatorsX=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
                         max_iter=max_iter, activation=activation)

min_child_weight=1.5
max_depth=5
gamma=2.5
colsample_bytree=0.8
subsample=0.8
n_estimators=300
threads = 4
estimators = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )


#estimators = BaggingClassifier(base_estimator=estimatorsXX,n_estimators=15,max_samples=0.65,
 #                            max_features=1.0, random_state=1,bootstrap=False, n_jobs=8)
#estimators = VotingClassifier( [('xgb', estimatorsX), ('mlp', estimatorsY)], weights=[1,1])
#estimators = GridSearchCV(estimator=estimatorsXX, param_grid=params, cv=10)





def poly(df, arr, param, polyName, polyLen):
    poly_ = param.fit_transform(df[arr])
    polynomials = pd.DataFrame(param
                                    .fit_transform(df[arr]),
                                    columns=[polyName+str(i) for i in range(polyLen)],index=df.index)
    #polynomials = polynomials.drop([polyName+'0'], axis=1)
    df = pd.concat([df, polynomials], axis=1)
    return df

def removeColumns(npArry, columnNames):
    li = list(npArry)
    for i in columnNames:
        li.remove(i)
    return np.array(li)


def powerAll(data_frame, exceptList,power):
    allColumns = data_frame.columns.values
    for i in allColumns:
        if(i not in exceptList ):
            data_frame[i] = data_frame[i]**power
 



#nothing
exceptList = []
power = 1

def formatFinal(df):
    dropList = ['fare','timediff','distance','meter_waiting','duration','fare',
            'meter_waiting_fare','additional_fare','meterMinfare','rush', 'nightHours',
            'drivemintime','fareDis','fareMin','man_fare']

    df = formatdata(df)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(df.mean())
    df = poly(df, polyList1, features_transformer1, 'poly1_', 9)
    df = poly(df, polyList2, features_transformer2, 'poly2_', 9)
    df = poly(df, polyList3, features_transformer3, 'poly3_', 10)
    df = poly(df, polyList4, features_transformer4, 'poly4_', 6)
    df = poly(df, polyList5, features_transformer5, 'poly5_', 10)
    df = poly(df, polyList6, features_transformer6, 'poly6_', 5)
    df = poly(df, polyList7, features_transformer7, 'poly7_', 6)
    df = poly(df, polyList8, features_transformer8, 'poly8_', 10)
    df = poly(df, polyList9, features_transformer9, 'poly9_', 9)
    df = poly(df, polyList10, features_transformer10, 'poly10_',6)
    df = poly(df, polyList11, features_transformer11, 'poly11_', 9)
    
    constant_filter = VarianceThreshold(threshold=0)
    df = pd.DataFrame(constant_filter.fit_transform(df),index=df.index)
    qconstant_filter = VarianceThreshold(threshold=0.01)
    df = pd.DataFrame(qconstant_filter.fit_transform(df),index=df.index)
    

    return df

X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=2
)

                    
numeric_cols = X_train.columns[X_train.dtypes != object].values

#powerAll(features_df, exceptList,1)
#exceptList = features_df.columns[features_df.dtypes == "object"].values
#polyList1 = ['meter_waiting_fare','fare']
polyList1 = ['rush','fare','timediff']
polyList2 = ['timediff','distance']
polyList3 = ['meter_waiting','duration']
polyList4 = ['timediff','fare']
polyList5 = ['distance','fare']
polyList6 = ['meter_waiting_fare','additional_fare']
polyList7 = ['allrush','fare']
polyList8 = ['meterMinfare','drivemintime']
polyList9 = ['fareDis','fareMin']
polyList10 = ['nightHours','rush']
polyList11 = ['man_fare','distance']

features_transformer1 = PolynomialFeatures(degree=2, include_bias=False,interaction_only=False)
features_transformer2 = PolynomialFeatures(degree=3, include_bias=False,interaction_only=False)
features_transformer3 = PolynomialFeatures(degree=3, include_bias=True,interaction_only=False)
features_transformer4 = PolynomialFeatures(degree=2, include_bias=True,interaction_only=False)
features_transformer5 = PolynomialFeatures(degree=3, include_bias=True,interaction_only=False)
features_transformer6 = PolynomialFeatures(degree=2, include_bias=False,interaction_only=False)
features_transformer7 = PolynomialFeatures(degree=2, include_bias=True,interaction_only=False)
features_transformer8 = PolynomialFeatures(degree=3, include_bias=True,interaction_only=False)
features_transformer9 = PolynomialFeatures(degree=3, include_bias=False,interaction_only=False)
features_transformer10 = PolynomialFeatures(degree=2, include_bias=True,interaction_only=False)
features_transformer11 = PolynomialFeatures(degree=3, include_bias=False,interaction_only=False)


X_train = formatFinal(X_train)
X_eval = formatFinal(X_eval)


allCols = X_train.columns.values


C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ValueError: unknown type str128

In [15]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=2
)
allCols = X_train.columns.values
preprocessor = ColumnTransformer(
    transformers = [
                
        ("scaler", StandardScaler(), allCols),
        ("norm", Normalizer(norm ='l2'), allCols)
    ],
    remainder = "drop"
)
          
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

full_pipeline.fit(X_train, y_train)
preds = full_pipeline.predict(X_eval)
y_preds = pd.DataFrame(
    preds,
    index = y_eval.index
)

f1 = f1_score(y_eval, y_preds)
print('f1',f1)
tn, fp, fn, tp = confusion_matrix(y_eval, y_preds).ravel()

print(tn, fp, fn, tp)
print(confusion_matrix(y_eval,y_preds))
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                                index_col="tripid")
test_features_df = formatdata(test_features_df)
powerAll(test_features_df, exceptList,power)

test_probas = full_pipeline.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)
print(test_preds.prediction.value_counts())

ValueError: Input contains infinity or a value too large for dtype('float64').

In [6]:
clf1 = ''
clf2 = ''
clf3 = ''
clf4 = ''
clf5 = ''
statesList = [3,6,42,2,1]
for i in statesList:    
    kf = KFold(n_splits=10, random_state=i, shuffle=True)


    kf.get_n_splits(features_df)
    test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                                   index_col="tripid")

    test_features_df = formatdata(test_features_df)
    powerAll(test_features_df, exceptList,power)

    bestscore = 0
    bestModelx = ""




    for train_index, test_index in kf.split(features_df):

        X_train, X_eval, y_train, y_eval = features_df.iloc[train_index], features_df.iloc[test_index],labels_df.iloc[train_index], labels_df.iloc[test_index]
        full_pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("estimators", estimators),
        ])
        full_pipeline.fit(X_train, y_train)
        preds = full_pipeline.predict(X_eval)
        y_preds = pd.DataFrame(
            preds,
            index = y_eval.index
        )
        test_probas = full_pipeline.predict(test_features_df)
        test_preds = pd.DataFrame(
            {'prediction':test_probas},
            index = test_features_df.index
        )
        f1 = f1_score(y_eval, y_preds)
        print(f1)
        if(f1 > bestscore):
            bestscore = f1
            bestModelx = full_pipeline


    
    test_probas1 = bestModelx.predict(test_features_df)
    test_preds1 = pd.DataFrame(
        {'prediction':test_probas1},
        index = test_features_df.index
    )
    if(i == 3):
        
        clf1 = pd.DataFrame(
            {'prediction':test_probas1},
            index = test_features_df.index
        )
        print(clf1.prediction.value_counts())
    elif(i==6):
        clf2 = pd.DataFrame(
            {'prediction':test_probas1},
            index = test_features_df.index
        )
        print(clf2.prediction.value_counts())
    elif(i==2):
        clf4 = pd.DataFrame(
            {'prediction':test_probas1},
            index = test_features_df.index
        )
        print(clf4.prediction.value_counts())
    elif(i==1):
        clf5 = pd.DataFrame(
            {'prediction':test_probas1},
            index = test_features_df.index
        )
        print(clf5.prediction.value_counts())
    else:
        clf3 = pd.DataFrame(
            {'prediction':test_probas1},
            index = test_features_df.index
        )
        print(clf3.prediction.value_counts())
    print('bestscore', bestscore)


0.9662053427743804
0.9710796915167096
0.9753600000000001
0.9723018147086915
0.9731800766283525
0.9730757047830219
0.9725984251968502
0.9712843168191859
0.974063400576369
0.9765970904490829
1    8125
0     451
Name: prediction, dtype: int64
bestscore 0.9765970904490829
0.9733547351524879
0.9730407523510971
0.9722747321991179
0.9739351557533376
0.9689677010766308
0.9712276214833759
0.971830985915493
0.9736424261670372
0.9773380146824131
0.969443550981023
1    8137
0     439
Name: prediction, dtype: int64
bestscore 0.9773380146824131
0.9741161616161617
0.9729729729729729
0.974784551548037
0.9722572509457756
0.9713375796178344
0.9679466835925103
0.9733033129623673
0.9704832954914044
0.9714828897338403
0.9773156899810964
1    8126
0     450
Name: prediction, dtype: int64
bestscore 0.9773156899810964
0.9717729146844276
0.9743589743589743
0.9746353836398224
0.9713375796178344
0.9716470213443771
0.9719987125844867
0.9728535353535354
0.9730586370839936
0.9739682539682539
0.9710238248551192
1   

In [7]:


print(clf4.equals(clf1))
print(clf1.prediction.value_counts())

False
1    8125
0     451
Name: prediction, dtype: int64


In [18]:
test_preds1 = clf1
print(test_preds1.prediction.value_counts())

test_preds2 = clf2
print(test_preds2.prediction.value_counts())

test_preds3 = clf3
print(test_preds3.prediction.value_counts())

test_preds4 = clf4
print(test_preds4.prediction.value_counts())

test_preds5 = clf5
print(test_preds5.prediction.value_counts())

final_preds = clf1

for index, row in test_preds1.iterrows():
    df1_row_value =  test_preds1.loc[index].prediction
    df2_row_value =  test_preds2.loc[index].prediction
    df3_row_value =  test_preds3.loc[index].prediction
    df4_row_value =  test_preds4.loc[index].prediction
    df5_row_value =  test_preds5.loc[index].prediction

    if(df1_row_value + df2_row_value + df3_row_value + df4_row_value + df5_row_value >= 3):
        final_preds.loc[index].prediction = 1
    else:
        final_preds.loc[index].prediction = 0
    
print(final_preds.prediction.value_counts())






NameError: name 'clf1' is not defined

In [17]:
#0.9828134945894335 state 3 k=10
#0.9816188326346339 state 6 k=10
final_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

NameError: name 'final_preds' is not defined

In [ ]:

solver = 'lbfgs'
alpha = 0.2
hidden_layer_sizes=(10,2 )
random_state=2
max_iter= 10000000
activation='tanh'
estimatorsX=MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state,
                         max_iter=max_iter, activation=activation)

min_child_weight=1.5
max_depth=5
gamma=2.5
colsample_bytree=0.8
subsample=0.8
n_estimators=200
threads = 4
estimators = XGBClassifier(min_child_weight=min_child_weight,max_depth=max_depth, 
                           gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample,
                           n_estimators=n_estimators, n_jobs=threads
                          )
params = { 'max_features': [0.8,0.85,0.9,0.95,1.0],'max_samples':[0.6,0.65,0.7,0.75,0.8]}

#estimatorsY = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=6,learning_rate=0.2,
 #                          n_estimators=1000,random_state=42)

#estimators = BaggingClassifier(base_estimator=estimatorsY,n_estimators=10,max_samples=0.65,
 #                            max_features=1.0, random_state=1,bootstrap=False)
#estimators = VotingClassifier( [('xgb', estimatorsX), ('mlp', estimatorsY)], weights=[1,1])
#estimators = GridSearchCV(estimator=estimatorsB, param_grid=params, cv=5)


In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=2
)
smt = SMOTE(sampling_strategy='all', random_state=None, k_neighbors=5, n_jobs=1)
svmus = SVMSMOTE(sampling_strategy='auto',  k_neighbors=10, random_state=42,
                                n_jobs=4, out_step=0.1)
smottomek = SMOTETomek(sampling_strategy ='all', random_state=2, n_jobs=1)

#X_train, y_train = svmus.fit_resample(X_train, y_train)
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

full_pipeline.fit(X_train, y_train)
preds = full_pipeline.predict(X_eval)
y_preds = pd.DataFrame(
    preds,
    index = y_eval.index
)

f1 = f1_score(y_eval, y_preds)
print(f1)

In [ ]:
#print(full_pipeline.named_steps['estimators'].best_params_, full_pipeline.named_steps['estimators'].best_score_)
print(f1)

In [ ]:
test_features_df = pd.read_csv(DATA_PATH / "test.csv", 
                               index_col="tripid")

test_features_df = formatdata(test_features_df)
powerAll(test_features_df, exceptList,power)


test_probas = full_pipeline.predict(test_features_df)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features_df.index
)

test_preds.to_csv(DATA_PATH / 'my_submission.csv', index=True)

print(confusion_matrix(y_eval, y_preds))
print(test_preds.prediction.value_counts())
print(f1)

In [ ]:
print(confusion_matrix(y_eval, y_preds))
print(test_preds.prediction.value_counts())
print(f1)

In [ ]:
#median
#0.9762502413593357
#1    8113
#0     463
#[[ 367  188]
 #[  58 5056]]

In [ ]:
#mean
#0.9754447022428461
#1    8124
#0     452
#[[ 370  185]
# [  69 5045]]

In [ ]:
#most_frequent
#0.9740121727369336
#1    8126
#0     450
#[[ 359  196]
# [  73 5041]]

In [ ]:
#constant
0.9746713070378964
#1    8123
#0     453
#[[ 366  189]
# [  73 5041]]